 <font size="6">Сегментация и детектирование</font>

# Задачи компьютерного зрения

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/2.png" width="600">

В ходе предыдущих занятий, мы рассмотрели задачу классификации изображений и различные модели, используемые при решении этой задачи, а также поняли, как применять их к реальным данным. Однако, компьютерное зрение решает не только задачу классификации, но и такие задачи как сегментация, детектирование, создание подписей к изображением и многие другие. 

Сегодня мы поговорим о первых двух упомянутых задачах.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/0.png" width="600">

Прежде чем говорить о способах решения задач компьютерного зрения, разберемся с форматами входных данных.

## Dataset COCO — Common Objects in COntext

Один из наиболее популярных датасатов содержащий данные для сегментации и детектирования. Он содержит более трёхсот тысяч изображений, большая часть из которых размечена и содержит следующую информацию:
- Категории
- Маски
- Ограничивающие боксы (*bounding boxes*)
- Описания (*captions*)
- Ключевые точки (*keypoints*)
- И многое другое

Формат разметки изображений, использованный в этом датасете, нередко используется и в других наборах данных. Как правило, он упоминается просто как "COCO format".

Загрузим датасет.

In [ ]:
import requests, zipfile, io

r = requests.get(
    "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

Для работы с датасетом используется пакет `pycocotools`.

[Подробнее о том как создать свой COCO датасет с нуля](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch).

In [ ]:
from pycocotools.coco import COCO

coco = COCO("annotations/instances_val2017.json")

Рассмотрим формат аннотаций на примере одной записи.

In [ ]:
catIds = coco.getCatIds(catNms=["cat"])  # cats IDs
print('class ID(cat) = %i' % catIds[0])

imgIds = coco.getImgIds(catIds=catIds)  # Filtering dataset by tag
print("All images: %i" % len(imgIds))

Рассмотрим метаданные.

In [ ]:
img_list = coco.loadImgs(imgIds[0])  # 1 example
img = img_list[0]
img

Посмотрим на изображение.

In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt

I = io.imread(img["coco_url"])
plt.axis("off")
plt.imshow(I)
plt.show()

Сконвертируем в PIL формат для удобства дальнейшей работы.

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

pil_img = coco2pil(img["coco_url"])
plt.axis("off")
plt.imshow(pil_img)
plt.show()

### Категории в COCO

Давайте посмотрим на примеры категорий в нашем датасете. Отобразим каждую 10ую категорию.

In [ ]:
cats = coco.loadCats(coco.getCatIds())  # loading categories
num2cat = {}  
print("COCO categories: ")
for cat in cats:
    num2cat[cat["id"]] = cat["name"]
    if cat["id"] in range(0, 80, 10):
        print(cat["id"], ":", cat["name"], end="   \n")

В датасете также есть категория **0**. Ее используют для обозначения класса фона.

Также существуют надкатегории. 

In [ ]:
print(f'cats[2]: {cats[2]}')
print(f'cats[3]: {cats[3]}')

nms = set([cat["supercategory"] for cat in cats])
print("COCO supercategories: \n{}".format("\n".join(nms)))

### Вернемся к метаданным

Помимо метаданных нам доступна разметка ([подробнее о разметке](https://cocodataset.org/#format-data)), давайте её загрузим и отобразим.

In [ ]:
annIds = coco.getAnnIds(imgIds=img["id"])
anns = coco.loadAnns(annIds)

plt.imshow(I)
plt.axis("off")
coco.showAnns(anns)
plt.show()

На изображении можно увидеть разметку пикселей изображения по классам. То есть, пиксели из объектов, относящихся к интересующим классам, приписываются к классу этого объекта. К примеру, можно увидеть объекты двух классов: "cat" и "keyboard". 

Давайте теперь посмотрим, из чего состоит разметка.

In [ ]:
def dump_anns(anns):
    for i, a in enumerate(anns):
        print(f"\n#{i}")
        for k in a.keys():
            if k == "category_id" and num2cat.get(a[k], None):
                print(k, ": ", a[k], num2cat[a[k]])  # Show cat. name
            else:
                print(k, ": ", a[k])

dump_anns(anns)

Заметим, что аннотация изображения может состоять из описаний нескольких объектов, каждое из которых содержит следующую информацию:
* `segmentation` - последовательность пар чисел ($x$, $y$), координат каждой из вершин "оболочки" объекта;
* `area` - площадь объекта;
* `iscrowd` - информация о том, находится в оболочке один объект или же несколько, но слишком много для пообъектной разметки (толпа людей, к примеру);
* `image_id` - идентификатор изображения, к которому принаделжит описываемый объект;
* `bbox` - *будет рассмотрен далее в ходе лекции*;
* `category_id` - идентификатор категории, к которой относится данный объект;
* `id` - идентификатор самого объекта.

Попробуем посмотреть на пример, в котором `iscrowd = True` .

In [ ]:
plt.rcParams["figure.figsize"] = (120, 60)

catIds = coco.getCatIds(catNms=["people"])
annIds = coco.getAnnIds(catIds=catIds, iscrowd=True)
anns = coco.loadAnns(annIds[0:1])

dump_anns(anns)
img = coco.loadImgs(anns[0]["image_id"])[0]
I = io.imread(img["coco_url"])
plt.figure(figsize=(10, 10))
plt.imshow(I)
coco.showAnns(anns)  # People in the stands
seg = anns[0]["segmentation"]
print("Counts", len(seg["counts"]))
print("Size", seg["size"])
plt.axis("off")
plt.show()

Используя методы из `pycocotools`, можно с лёгкостью преобразовать набор вершин "оболочки" сегментируемого объекта в более удобный для отображения вид - в маску объекта.

In [ ]:
import numpy as np

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)
msk = np.zeros(seg["size"])

fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(10, 10))

i = 0
for row in range(4):
    for col in range(4):
        msk = coco.annToMask(anns[i])
        ax[row, col].imshow(msk, cmap = 'gray')
        ax[row, col].set_title(num2cat[anns[i]["category_id"]])
        ax[row, col].axis("off")
        i += 1
        
plt.show()

В некоторых случаях, попиксельная разметка изображения может быть избыточной - к примеру, в случае если необхдимо посчитать количество человек на изображении, достаточно просто каким-то образом промаркировать каждого из них, после чего посчитать количество наших "отметок". Одним из вариантов маркировки является "обведение" объекта рамкой (`bounding boxes`), внутри которой он находится. Такая информация об объектах также сохранена в аннотациях формата COCO.

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)

RGB_img = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)

for i in range(len(anns)):
    x, y, width, heigth = anns[i]["bbox"]
    x, y, width, heigth = int(x), int(y), int(width), int(heigth)
    if anns[i]["category_id"] == 1: # person
        color = (255, 255, 255)
    if anns[i]["category_id"] == 40: # glove
        color = (0, 255, 0)
    RGB_img = cv2.rectangle(RGB_img, (x, y), (x + width, y + heigth), color, 2)
cv2_imshow(RGB_img)

#### $\color{brown}{\text{Дополнительная информация}}$ 
### Еще более глубокое понимание разметки

Что такое [run-length encoding - RLE](https://en.wikipedia.org/wiki/Run-length_encoding)

[Видео-разбор](https://www.youtube.com/watch?v=h6s61a_pqfM)

# Семантическая сегментация (*Semantic segmentation*)


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/1.png" width="800">

Постановка задачи:

Предсказать класс для каждого пикселя.

Входные данные маска: 

[ x,y - > class_num ] 

Выходные данные маска:

[ x,y - > class_num ] 


## Способы предсказания класса для каждого пикселя

### a) Наивный

Простейшим вариантом решения является использование так называемого "скользящего окна" - последовательное рассмотрение фрагментов изображения. В данном случае, интересующими фрагментами будут небольшие зоны, окружающие каждый из пикселей изображения. К каждому из данных фрагментов применяется свёрточная нейронная сеть, предсказывающая, к какому классу относится центральный пиксель.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/3.png" width="600">

### б) Разумный

Изменить архитектуру используемой модели и убрать линейный слой в конце сети, сделав сеть полносвёрточной. При этом, важно сохранить пространственный размер (ширину и высоту) промежуточных и конечного представлений. Сделав в конечном представлении количество каналов равным количеству классов, можно использовать значения внутри каждой из карт активаций, как ненормированное значение вероятности отношения каждого из пикселей к тому или иному классу. 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/4.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/5.png" width="500">

В лекции про сверточные сети мы говорили о том, что свертку 1x1 можно рассматривать как полносвязанный слой. Именно так она и будет использоваться при этом способе сегментации, чтобы достичь необходимого количества каналов в конечном представлении.

Проблемы:
- нужно большое рецептивное поле, следовательно много слоев ($L$ раз свёртка $3\times3$ $\to$ рецептивное поле $(1+2L)\times(1+2L)$);
- очень медленно работает на полноразмерных картах активации.

### в) Эффективный

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/6.png" width="800">

В традиционной структуре свёрточной сети присутствует сжатие пространственных размеров, что позволяет стремительнее увеличивать рецептивные поля нейронов на более глубоких уровнях. Благодаря этому, нейросети может извлекать информацию о всё более сложных зрительных образах, встречающихся на изображениях. Данные признаки могут быть полезны при предсказнии принадлежности пикселей к классам, однако проблема заключается в том, что карты активаций меньше размера исходного изображения. Чтобы решить данную проблему, нейросеть завершают разжимающим блоком, позволяющим постепенно увеличивать карты активаций.

## Автокодировщик

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/7.png" width="500">

Такая архитектура довольно популярна и применяется не только для сегментации: 

- сглаживание шума;
- снижение размерности $\to$ вектор-признак;
- генерация данных.

Этому будет посвящена одна из следующих лекций, сейчас же мы детально рассмотрим разжимающий блок.

### Изменение размеров изображений 

In [ ]:
import warnings 
warnings.filterwarnings("ignore")

import torch
import seaborn as sns
import matplotlib.pyplot as plt

# upsample layers
upsample_nn = torch.nn.Upsample(mode='nearest', scale_factor=2) # nearest
upsample_bl = torch.nn.Upsample(mode='bilinear', scale_factor=2) # bilinear

# define dummy tensor
a = torch.tensor([[1,3,0,1],[3,3,3,7],[8,1,8,7],[6,1,1,1]]).float() 
a = a[None, None,:] # add axis

interp_nn = upsample_nn(a)
interp_bl = upsample_bl(a)

# plot result
fig, ax = plt.subplots(ncols=3, figsize=(15,5), sharex=True, sharey=True);
sns.heatmap(a[0][0], annot=True, ax=ax[0], lw=1, cbar=False);
sns.heatmap(interp_nn[0][0], annot=True, ax=ax[1], lw=1, cbar=False);
sns.heatmap(interp_bl[0][0], annot=True, ax=ax[2], lw=1, cbar=False);
for a in ax:
    a.axis('off')

ax[0].set_title('Raster dataset')
ax[1].set_title('Nearest neighbor interpolation')
ax[2].set_title('Bilinear interpolation')
plt.show()

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/8-1.png" width="600">

Билинейная интерполяция рассматривает квадрат $2\times2$ известных пикселя, окружающих неизвестный. В качестве интерполированного значения используется взвешенное среднее этих четырёх пикселей. 

Отметим, что чем ближе интерполируемое значение находится к одному из известных пикселей, тем больший вес будет у значения этого пикселя при подсчёте взвешенной суммы. 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/8.png" width="550">

Периодически, столь небольшой контекст ($2\times2$ известных пикселя) может быть не достаточен для сохранения важной информации о распределении цветов в окружающих пикселях (пример будет приведён далее). В таких случаях, можно попробовать использовать большее количество окружающих пикселей для интерполяции - к примеру, $4\times4$. Такая интерполяция называется бикубической и имеет значительно более сложную формулу, чем билинейная, однако основной приницп остаётся тем же - чем ближе пиксель к пикселю с известным значением, тем больше "вес" последнего при интерполяции.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/9.png" width="550">

### Upsample в Pytorch

Для увеличения разрешения изображения, в PyTorch используется класс `Upsample`. В нём доступны все упомянутые методы интерполяции, а также трилинейная интерполяция - аналог билинейной интерполяции, используемый для работы с трёхмерными пространственными данными (к примеру, видео). Отметим, что методы интерполяции можно использовать как для самих изображений или видео, так и для карт активаций, появляющихся в процессе работы нейросети. Таким образом, мы можем использовать `Upsample` внутри нашего разжимающего блока.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/10.png" width="800">

[[doc] nn.Upsample](https://pytorch.org/docs/stable/generated/torch.nn.Upsample.html)

[[doc] nn.functional.interpolate](https://pytorch.org/docs/stable/generated/torch.nn.functional.interpolate.html?highlight=interp#torch.nn.functional.interpolate)

In [ ]:
import torch
from torch import nn
import torchvision.transforms.functional as TF
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from PIL import Image


def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

def upsample(pil, ax, mode="nearest"):
    tensor = TF.to_tensor(pil)
    upsampler = nn.Upsample(scale_factor=2, mode=mode)
    tensor_128 = upsampler(tensor.unsqueeze(0))
    img_128 = TF.to_pil_image(tensor_128.squeeze()).convert("RGB")
    ax.imshow(img_128)
    ax.set_title(mode)
    ax.set_xlim(0, 20 * 2)
    ax.set_ylim(20 * 2, 0)
    ax.axis("off")


pic = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")
pil_64 = pic.resize((64, 64))

fig, ax = plt.subplots(ncols=4, figsize=(15, 5))
ax[0].imshow(pil_64)
ax[0].set_title("Resized image")
ax[0].set_xlim(0, 20)
ax[0].set_ylim(20, 0)
ax[0].axis("off")


upsample(pil_64, mode="nearest", ax=ax[1])
upsample(pil_64, mode="bilinear", ax=ax[2])
upsample(pil_64, mode="bicubic", ax=ax[3])
plt.show()

Обратите внимание, что в данном случае каждое из пространственных измерений изображения увеличилось в 2 раза, но при необходимости возможно использовать увеличение в иное количество раз!

### MaxUnpooling

Помимо свёртки, на этапе снижения размерности также используются слои pooling'а. Наиболее популярным вариантом является maxpooling, сохраняющий информацию о значении наибольшего элемента внутри сегментов. Для того чтобы обратить данную операцию субдискретизации, был предложен MaxUnpooling слой.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/11-1.png" width="650">

Данный слой требует от max pooling'а сохранения индексов максимальных элементов внутри сегментов - при обратной операции, максимальное значение приписывается тому же элементу сегмента, в котором был максимальный элемент сегмента до соответствующей субдискретизации. Соответственно, каждому слою MaxUnpooling должен соответствовать слой субдискретизации, что визуально можно представить следующим образом:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/11-2.png" width="650">

[Документация к MaxPool2d](
https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)

[Документация к MaxUnpool2d](
https://pytorch.org/docs/stable/generated/torch.nn.MaxUnpool2d.html?highlight=unpooling)

In [ ]:
import torch
from torch import nn
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt


def tensor_show(tensor, title="", ax=ax):
    img = TF.to_pil_image(tensor.squeeze()).convert("RGB")
    ax.set_title(title + str(img.size))
    ax.imshow(img)


pool = nn.MaxPool2d(kernel_size=2, return_indices=True)  # False by default(get indexes to upsample)
unpool = nn.MaxUnpool2d(kernel_size=2)

pil = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")

fig, ax = plt.subplots(ncols=5, figsize=(20, 5), sharex=True, sharey=True)

ax[0].set_title("original " + str(pil.size))
ax[0].imshow(pil)
tensor = TF.to_tensor(pil).unsqueeze(0)
print("Orginal shape", tensor.shape)

# Downsample
tensor_half_res, indexes1 = pool(tensor)
tensor_show(tensor_half_res, "1/2 down ", ax=ax[1])

tensor_q_res, indexes2 = pool(tensor_half_res)
tensor_show(tensor_q_res, "1/4 down ", ax=ax[2])
print("Downsample shape", indexes2.shape)

# Upsample
tensor_half_res1 = unpool(tensor_q_res, indexes2)
tensor_show(tensor_half_res1, "1/2 up ", ax=ax[3])


tensor_recovered = unpool(tensor_half_res1, indexes1)
tensor_show(tensor_recovered, "full size up ", ax=ax[4])
print("Upsample shape", tensor_recovered.shape)
plt.show()

Зачем нужен pad?


In [ ]:
import torch
import torch.nn.functional as F
import seaborn as sns
torch.manual_seed(42)


fig, ax = plt.subplots(ncols=2, figsize=(10, 5), sharex=True, sharey=True)
array = torch.ones((24, 24), dtype=int)
sns.heatmap(array, annot=True, fmt="d", ax=ax[0], cbar=False, vmin=0, vmax=1)
print("Array size:", array.size())

array_padded = F.pad(array, pad=[4, 4])
sns.heatmap(array_padded, annot=True, fmt="d", ax=ax[1], cbar=False)
print("Array size with padding:", array.size())
plt.show()

### Transpose convolution

Способы восстановления пространственных размерностей, которые мы рассмотрели, не содержали обучаемых параметров. Для повышения пространственного разрешения можно использовать аналог операции свёртки - транспонированную свёртку. 

<center><img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/13.png" width="700"></center>
<center><em>Обычная свертка.</em></center>

Отличие этого метода от свёртки заключается в том, что вместо перемножения сегмента изображения и фильтра с целью получить скаляр, используется домножение скаляра на фильтр с целью получения некого сегмента создаваемой карты активаций. Причём в зонах, где полученные значения "накладываются" друг на друга, будет найдена их сумма.

<center><img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/14.png" width="700">
</center>
<center><em>Upsample/transpose convolution.</em></center>

<center><img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/16.png" width="500"></center>

[Блог-пост про 2d свертки с помощью перемножения матриц](https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544)

[Документация к ConvTranspose2d](
https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html?highlight=transpose#convtranspose2d)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/15.png" width="800">

Применяет 2D-транспонированный оператор свертки к входному изображению, состоящему из нескольких входных плоскостей.
Этот модуль можно рассматривать как градиент Conv2d по отношению к его входу. Это также известно, как свертка с частичным шагом или деконволюция (последнее - это не правильный термин, что-то вроде "Силиконовая долина").

In [ ]:
import torch
from torch import nn
import seaborn as sns

input = torch.randn(1, 16, 16, 16) # define dummy input
print('Original size', input.shape)

downsample = nn.Conv2d(16, 16, 3, stride=2, padding=1) # define downsample layer
upsample = nn.ConvTranspose2d(16, 16, 3, stride=2, padding=1) # define upsample layer

# let`s downsample and upsample input
with torch.no_grad(): 
  output_1 = downsample(input)
  print("Downsampled size", output_1.size())

  output_2 = upsample(output_1, output_size = input.size())
  print("Upsampled size", output_2.size())

# plot results
fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=True, sharey=True)
sns.heatmap(input[0, 0, :, :], ax=ax[0], cbar=False, vmin=-2, vmax=2)
ax[0].set_title("Input")
sns.heatmap(output_1[0, 0, :, :], ax=ax[1], cbar=False, vmin=-2, vmax=2)
ax[1].set_title("Downsampled")
sns.heatmap(output_2[0, 0, :, :], ax=ax[2], cbar=False, vmin=-2, vmax=2)
ax[2].set_title("Upsampled")
plt.show()

## U-Net: Convolutional Networks for Biomedical Image Segmentation

Популярная архитектура для сегментации. Изначально была предложена в статье [U-Net: Convolutional Networks for Biomedical Image Segmentation (Ronneberger et al., 2015)](https://arxiv.org/abs/1505.04597) для анализа  медицинских изображений.

<center><img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-20.png" width="700"></center>
<center><em>Архитектура U-Net (Ronneberger et al., 2015).</em></center>

[Реализация на PyTorch](https://github.com/milesial/Pytorch-UNet)

[U-Net на PyTorch Hub](https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/)

[Блог-пост разбор](https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5)

Стоит обратить особое внимание на серые стрелки на схеме: они соответствуют операции конкатенации копий ранее полученных карт активаций, по аналогии с DenseNet. Чтобы это было возможно, необходимо поддерживать соответствие между размерами карт активаций в процессах снижения и повышения пространственных размерностей. Для этой цели, изменения размеров происходят только при операциях `MaxPool` и `MaxUnpool` - в обоих случаях, в два раза. 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/18.png" width="650">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/17.png" width="400">

После upsample блоков ReLU не используется.

# Мультиклассовая сегментация

U-Net используется для одноклассовой сегментации, однако в некоторых случаях может понадобиться предсказать принадлежность пикселей к одному из нескольких классов. В таких случаях, как правило, используются иные модели, архитектуры некоторых из которых мы сейчас рассмотрим.

## Обзор Fully Convolutional Network(2014)  

Fully Convolutional Network
для того что бы не было путаницы с Fully Connected Network
последние именуют MLP (Multi Layer Perceptron)

Примеры реализации: [1](https://pytorch.org/hub/pytorch_vision_fcn_resnet101/) и [2](https://pytorch.org/vision/stable/models.html#semantic-segmentation)


Предобученная модель была обучена на части датасета COCO train2017 (на 20 категориях, представленных так же в датасете  Pascal VOC). Использовались следующие классы:

`['__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']`

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/l12out.png" width="500">

[FCN — Fully Convolutional Network (Semantic Segmentation)](https://towardsdatascience.com/review-fcn-semantic-segmentation-eb8c9b50d2d1)

Работает довольно просто. Берем любой *back-bone* (например `ResNet50` или `VGG16`) и прикручиваем слой `upsample` до нужных нам размеров в конце. Посмотрим, как оно работает:

In [ ]:
import torch
import random
import numpy as np
# fix seeds
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Compute on cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torchvision
from torch import nn
from torchvision import transforms
import torchvision.transforms.functional as TF
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from PIL import Image


def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


# load resnet50
fcn_model = torchvision.models.segmentation.fcn_resnet50(
    pretrained=True, num_classes=21
)

classes = ["__background__", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor",
           ]

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # ImageNet
    ]
)

pil_img = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")
input_tensor = transform(pil_img)

with torch.no_grad():
    output = fcn_model(input_tensor.unsqueeze(0))  

Возвращаются 2 массива

* out - каждый пиксель отражает ненормированную вероятность, соответствующую предсказанию каждого класса.

* aux - содержит значения *auxillary loss* на пиксель. На инференсе `output['aux']` бесполезный.

In [ ]:
print("output keys: ", output.keys())  # Ordered dictionary
print("out: ", output["out"].shape, "Batch, class_num, h, w")
print("aux: ", output["aux"].shape, "Batch, class_num, h, w")

output_predictions = output["out"][0].argmax(0)  # for first element of batch
print(f'output_predictions: {output_predictions.shape}')

fig = plt.figure(figsize=(10, 10))
plt.imshow(pil_img)
plt.axis("off")
plt.show()

indexes = output_predictions

# plot all classes predictions
fig, ax = plt.subplots(nrows=4, ncols=5, figsize=(10, 10))
i = 0 # counter
for row in range(4):
    for col in range(5):
        mask = torch.zeros(indexes.shape)
        mask[indexes == i] = 255
        ax[row, col].set_title(classes[i])
        ax[row, col].imshow(mask)
        ax[row, col].axis("off")
        i += 1

plt.show()

## Обзор DeepLabv3+(2018)

DeepLab - семейство моделей для сегментации, значительно развивавшееся в течение четырёх лет. Основой данного рода моделей является использование **atros (dilated) convolutions** и, начиная со второй модели, **atros spatial pyramid pooling**, опирающейся на **spatial pyramid pooling**.

[Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation (Chen et al., 2018)](https://arxiv.org/abs/1802.02611v3)

[Реализация на PyTorch](https://pytorch.org/vision/stable/models.html#deeplabv3)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/19.png" width="800">

### Spatial pyramid pooling (SPP) layer

[Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition (He et al., 2014)](https://arxiv.org/abs/1406.4729)

**Spatial Pyramid Pooling (SPP)** - это *pooling* слой, который устраняет ограничение фиксированного размера сети, т.е. CNN не требует входного изображения фиксированного размера. В частности, мы добавляем слой SPP поверх последнего сверточного слоя. 

Слой SPP объединяет признаки и генерирует выходные данные фиксированной длины, которые затем поступают в MLP (или другие классификаторы). Другими словами, мы выполняем некоторую агрегацию информации на более глубоком этапе иерархии сети (между сверточными слоями и полностью связанными слоями), чтобы избежать необходимости обрезать или деформировать изображение в начале.

<center><img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-23.png" width="700"></center>
<center><em>Схема SPP (He et al., 2014).</em></center>


В pytorch нет отдельного модуля, но результат может быть получен за счет применения нескольких AdaptiveMaxPool2d с разными размерами выходов.

### Atros (Dilated) Convolution

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/16.png" width="650">

**Dilated convolution** (расширенные свертки) - это тип свертки, который "раздувает" ядро, вставляя отверстия между элементами ядра. Дополнительный параметр (скорость расширения, **dilation**) указывает, насколько сильно расширяется ядро.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-26-1.png" width="800">

[[doc] nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)

In [ ]:
import seaborn as sns

# Atros example
with torch.no_grad():
    # define dummy input
    input = torch.tensor([[[[1, 1, 1],
                            [1, 1, 1],
                            [1, 1, 1]]]], dtype=torch.float)
    
    # define conv layer, dilation = 1
    conv = nn.Conv2d(1, 1, kernel_size=2, dilation=1, bias=False)
    # define kernel weights
    conv.weight = nn.Parameter(torch.tensor([[[[2, 2],
                                               [2, 2]]]], dtype=torch.float))
    output = conv(input)

# plot results
fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=False, sharey=False)
sns.heatmap(
    input[0][0], ax=ax[0], annot=True, fmt=".0f", cbar=False, vmin=0, vmax=8, linewidths=1
)
sns.heatmap(
    conv.weight.detach()[0, 0, :, :],
    ax=ax[1],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)
sns.heatmap(
    output[0, 0, :, :],
    ax=ax[2],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)

ax[0].set_title("Input \nshape: "+str(input.shape))
ax[1].set_title("Kernel \nshape: "+str(conv.weight.shape))
ax[2].set_title("Output \nshape: "+str(output.shape))
fig.suptitle("Dilation = 1", y=1.05)
plt.show()

In [ ]:
# change dilation to 2
conv = nn.Conv2d(1, 1, kernel_size=2, dilation=2, bias=False)  # Fell free to change dilation
conv.weight = nn.Parameter(torch.tensor([[[[2, 2],
                                           [2, 2]]]], dtype=torch.float))

output = conv(input)

# plot results
fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=False, sharey=False)
sns.heatmap(
    input[0][0], ax=ax[0], annot=True, fmt=".0f", cbar=False, vmin=0, vmax=8, linewidths=1
)
sns.heatmap(
    conv.weight.detach()[0, 0, :, :],
    ax=ax[1],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)
sns.heatmap(
    output[0, 0, :, :].detach(),
    ax=ax[2],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)

ax[0].set_title("Input \nshape: "+str(input.shape))
ax[1].set_title("Kernel \nshape: "+str(conv.weight.shape))
ax[2].set_title("Output \nshape: "+str(output.shape))
fig.suptitle("Dilation = 2", y=1.05)
plt.show()

In [ ]:
# change input tensor
input = torch.tensor([[[[0, 1, 0],
                        [1, 1, 1],
                        [0, 1, 0]]]], dtype=torch.float)
output = conv(input)

#plot results
fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=False, sharey=False)
sns.heatmap(
    input[0][0], ax=ax[0], annot=True, fmt=".0f", cbar=False, vmin=0, vmax=8, linewidths=1
)
sns.heatmap(
    conv.weight.detach()[0, 0, :, :],
    ax=ax[1],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)
sns.heatmap(
    output[0, 0, :, :].detach(),
    ax=ax[2],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)

ax[0].set_title("Input \nshape: "+str(input.shape))
ax[1].set_title("Kernel \nshape: "+str(conv.weight.shape))
ax[2].set_title("Output \nshape: "+str(output.shape))
fig.suptitle("Dilation = 2", y=1.05)
plt.show()

### $\color{brown}{\text{Дополнительная информация}}$ 

#### IoU - оценка точности

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-52.png" width="700">

[Intersection over Union](http://datahacker.rs/deep-learning-intersection-over-union/)

#### Pixel-wise cross entropy loss


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/24.png" width="800">

In [ ]:
from torch import nn
import torch

one_class_out = torch.randn(1, 1, 32, 32)
one_class_target = torch.randn(1, 1, 32, 32)

# https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
cross_entropy = nn.CrossEntropyLoss()

# https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html
bce_loss = nn.BCEWithLogitsLoss()
loss = bce_loss(one_class_out, one_class_target)
print("BCE", loss)


two_class_out = torch.randn(1, 2, 32, 32)
two_class_target = torch.randint(1, (1, 32, 32))

print(two_class_out.shape)
print(two_class_target.shape)

loss = cross_entropy(two_class_out, two_class_target)

print("Cross entropy loss", loss)

#### DiceLoss

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/dice.jpeg" width="800">

[Блог-пост про семантическую сегментацию](https://www.jeremyjordan.me/semantic-segmentation/)

In [ ]:
class BinaryDiceLoss(nn.Module):
    """Soft Dice loss of binary class
    Args:
        p: Denominator value: \sum{x^p} + \sum{y^p}, default: 2
        predict: A tensor of shape [N, *]
        target: A tensor of shape same with predict
       Returns:
        Loss tensor

    """

    def __init__(self, p=2, epsilon=1e-6):
        super().__init__()
        self.p = p  # pow degree
        self.epsilon = epsilon

    def forward(self, predict, target):
        predict = predict.flatten(1)
        target = target.flatten(1)

        # https://pytorch.org/docs/stable/generated/torch.mul.html
        num = torch.sum(torch.mul(predict, target), dim=1) + self.epsilon
        den = torch.sum(predict.pow(self.p) + target.pow(self.p), dim=1) + self.epsilon
        loss = 1 - 2 * num / den

        return loss.mean()  # over batch


criterion = BinaryDiceLoss()
output = torch.tensor([[[1, 1, 1], [1, 1, 1], [1, 1, 1]]], dtype=torch.float)


target = torch.tensor([[[1, 1, 1], [1, 1, 1], [1, 1, 1]]], dtype=torch.float)

soft_loss = criterion(output.unsqueeze(0), target.unsqueeze(0))
print("Loss", soft_loss)

# Детектирование (Object detection)


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/0.png" width="650">

Детектирование - задача компьютерного зрения, в которой модели требуется восстановить информацию об ограничивающих прямоугольниках объектов, относящихся к различным классам. 

Например:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-05.png" width="700">


Задачу семантической сегментации мы решали через классификацию. Для детектирования разумно использовать регрессию.

В случае для одного объекта можно обучить модель предсказывать числа:

* координаты центра + ширину и высоту
* координаты правого верхнего и левого нижнего углов
* координаты вершин многоугольника ...



In [ ]:
# fix random_seed
import torch
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Compute on cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from IPython.display import clear_output
from torchvision.models import resnet18
from torch import nn

# load pretrained model
resnet_detector = resnet18(pretrained=True)
clear_output()
# Change "head" to predict coordinates (x1,y1 x2,y2)
resnet_detector.fc = nn.Linear(resnet_detector.fc.in_features, 4)  # x1,y1 x2,y2

criterion = nn.MSELoss()

# This is a random example. Don't expect good results
input = torch.rand((1, 3, 224, 224))
target = torch.tensor([[0.1, 0.1, 0.5, 0.5]])  # x1,y1 x2,y2 or x,y w,h
print(f'Target: {target}')
output = resnet_detector(input)
loss = criterion(output, target)
print(f'Output: {output}')
print(f'Loss: {loss}')

[Recent Progress in Appearance-based Action Recognition (Humphreys et al., 2020)](https://arxiv.org/abs/2011.12619)

На прошлом семинаре мы упоминали про модели, которые ищут ключевые точки на лице человека (MTCNN). Можно использовать тот же подход для поиска любых точек.

<center><img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-07.png" width="700"></center>
<center><em>Примеры предсказывания точек (Humphreys et al., 2020)</em></center>


Начнем с ситуации, когда объект один.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/26.png" width="650">

Вводные:
- изображение
- координаты границ прямоугольника boundind box
(x,y,w,h)
- класс объекта

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/27.png" width="650">

Подобно задаче сегментации, мы можем попробовать создать решение на основе предобученного классификатора, на основе которого мы попробуем создать модель, решающую задачу детекции. Для этой цели модели придётся одновремено улучшать свои предсказания по определению расположения объекта (координат ограничивающего прямоугольника) и по классу найденного объекта. Чтобы при обучении модели скомбинировать две функции потерь, перед поиском градиента находится сумма данных ошибок и веса обновляются уже с учётом неё.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/29.png" width="650">

### $\color{brown}{\text{Дополнительная информация}}$ 

##### Regression loss

$\displaystyle\mathrm{MSE} = \frac{\sum^n_{i=1}\left(y_i-y_i^p\right)^2}{n}$ - L2/ MSE/ Mean Squared Error/ Среднеквадратичная ошибка 


$\displaystyle\mathrm{MAE} = \frac{\sum^n_{i=1}\left|y_i-y_i^p\right|}{n}$ - L1/ MAE/ Mean Absolute Error/ Средняя ошибка


$\displaystyle\
    L_{\delta}(y, f(x))=\left\{
                \begin{array}{ll}
                  \frac{1}{2}\left(y-f\left(x\right)\right)^2 \qquad &\mathrm{for}\  |y-f(x)| \leq \delta\\
                  \delta|y-f(x)|-\frac{1}{2}\delta^2 \qquad &\mathrm{otherwise}
                \end{array}
              \right.
  $ - Huber Loss/ Smooth Mean ABsolute Error/ Функция потерь Хьюбера


$\displaystyle L\left(y,y^p\right)=\sum_{i=1}^n log\left(cosh\left(y^p_i-y_i\right)\right)$ - Log-Cosh Loss/ Логарифм гиперболического косинуса

 <img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-12.png" width="700">

[5 Regression Loss Functions All Machine Learners Should Know](https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0)

MAE vs MSE

 <img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-13.png" width="700">

Huber vs Log-cos

##### Multitask loss

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-14.png" width="500">

[Multi-Task Learning Using Uncertainty to Weigh Losses for Scene Geometry and Semantics](https://arxiv.org/pdf/1705.07115.pdf)

[Пример реализации MultiTask learning](https://github.com/Hui-Li/multi-task-learning-example-PyTorch/blob/master/multi-task-learning-example-PyTorch.ipynb)


## Детектирование нескольких объектов

На практике, приходится чаще сталкиваться с одновременным детектированием нескольких объектов, нежели с детектированием одного объекта. Это может показаться проблемой, поскольку архитектура нейросети фиксирована, в связи с чем фиксировано и количество предсказываемых ответов, однако если для одного объекта мы предсказываем $k$ значений (к примеру, для предсказания `(x, y, w, h)`, $k = 4$), то для изображения с $N$ объектами будет необходимо предсказать $N \cdot k$ значений, причём значение $N$ может меняться от изображения к изображению, что вызывает дополнительные трудности.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-042.gif" width="700">

[Stanford University CS231n: Detection and Segmentation](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

### a) Наивный способ решения: скользящее окно

Одним из вариантов решения данной задачи является применение классификатора ко всем возможным местоположениям объектов. В данном случае, классификатор решает, есть ли на выбранном фрагменте изображения один из интересующих нас объектов или же нет - тогда фрагмент относится к классу "фон". 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-044.gif" width="700">




Проблемой данного подхода является то, что приходится применять классификатор к огромному количеству различных фрагментов, что крайне дорого с точки зрения вычислений.

### b) Эвристика

Вместо того чтобы применять классификатор "наобум", можно для начала выбрать те области изображения, в которых вероятность нахождения объекта наиболее высока и запускать классификатор лишь для них. Данные области называются **regions of interest**, сокращённо - **ROI**. 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-049.png" width="700">



- Найдите области “больших” (blobby) изображений, которые, вероятно, будут содержать объекты
- Относительно быстрый запуск; Selective Search дает 2000 предложений за несколько секунд на CPU

### Selective search

Selective search - известный алгоритмический метод поиска **ROI**, целью которого было быстрое обнаружение объектов разного размера. Для этого использовалось последовательное объединение потенциальных **ROI** на основе комбинации четырёх метрик сходств предполагаемых областей интереса. 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-18.png" width="800">



[Статья про Selective Search](http://www.huppelen.nl/publications/selectiveSearchDraft.pdf)

Возвращает порядка 2000 прямоугольников для изображения. С таким количеством уже можно работать.

### R-CNN - Region CNN 
Построена по такому принципу:

- на изображении ищутся ROI 
- для каждого делается resize 
- каждый ROI обрабатывается сверточной сетью, которая предсказывает класс объекта, который в него попал


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-055.png" width="700">

Кроме класса модель предсказывает смещения для каждого bounding box

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-20.png" width="700">

### NMS

Теперь возникает другая проблема: в районе объекта алгоритм генерирует множество ограничивающих прямоугольников (bounding box), которые частично перекрывают друг друга.

Чтобы избавиться от них используется другой алгоритм
Non maxima suppression.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/31.png" width="650">

Его задача избавиться от bbox, которые накладаваются на истинный

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/32.png" width="650">

Для оценки схожести обычно используется метрика IoU, а значение IoU, при котором bbox считаются принадлежащими одному объекту, является гиперпараметром (часто 0.5).

### Soft NMS

Другой способ решения данной задачи - Soft NMS. В процессе работы, он не удаляет наименее вероятные из перекрывающихся результатов детекции. Вместо этого, он, как и NMS, оставляет неизменным наиболее вероятный ограничивающий прямоугольник $\mathcal{M}$, уменьшая вероятность обнаружения объекта внутри некого прямоугольника $b_i$ при увеличении значения $\text{iou}$ прямоугольников $\mathcal{M}$ и $b_i$. Простым вариантом является домножение вероятности $s_i$ на $(1 - \text{iou}(\mathcal{M}, b_i)$.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/33.png" width="650">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/34-1.png" width="800">

[[doc] torchvision.ops.nms](https://pytorch.org/vision/stable/ops.html#torchvision.ops.nms)

### Fast R-CNN

Проблемой описанного выше подхода является скорость.
Так как мы вынуждены применять CNN порядка 2000 раз (в зависимости от эвристики, которая генерирует ROI)


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-26.png" width="800">

И решением является поиск ROI не на самом изображении, а на карте признаков, полученной после обработки всего изображения CNN. В таком случае большая часть сверток выполняется только один раз.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-27.png" width="700">

Это радикально ускоряет процесс

### ROI Pooling

Появляется новая задача - 'resize' ROI на карте признаков.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/35.png" width="650">

[Документация Roi Pooling](https://pytorch.org/vision/stable/ops.html#torchvision.ops.roi_pool)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/36-1.png" width="800">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-077.png" width="700">

Скорость работы CNN снизилась и теперь узким местом становится эвристика для поиска ROI

### Faster R-CNN


Добавим Region Proposal Сеть (RPN) для прогнозирования предложения от функций



Обучается с 4 лоссами:
1. RPN классифицирует объект/не объект
2. Координаты блока регрессии RPN
3. Итоговая классификационная оценка (классы объекта)
4. Окончательные координаты бокса 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/37.png" width="650">

**Идея: пусть сеть сама предсказывает ROI по карте признаков**

Для обучения требуется посчитать 4 loss.

### Region proposal network

Карта признаков имеет фиксированные и относительно небольшие пространственные размеры (например 20x15)


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/35.png" width="800">

Поэтому можно вернуться к идее скользящего окна которая была отвергнута в самом начале.

При этом можно использовать окна нескольких форм

Предсказываются два значения:

* вероятность того что в ROI находится объект
* смещения

Сама сеть при этом может быть очень простой:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/41.png" width="400">

В результате скорость увеличивается почти в 10 раз

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-086.png" width="700">

[Модель на PyTorch](https://pytorch.org/vision/stable/models.html#faster-r-cnn)

In [ ]:
import torchvision
# load model
fr_rcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    pretrained=True, progress=True, num_classes=91, pretrained_backbone=True
)
fr_rcnn.eval()

Загрузим данные

In [ ]:
from pycocotools.coco import COCO
import requests
import zipfile
import io

# load data
r = requests.get(
    "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
coco = COCO("annotations/instances_val2017.json")

In [ ]:
from PIL import Image
from io import BytesIO

def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

catIds = coco.getCatIds(catNms=["person", "bicycle"]) # get category IDs
# person and bicycle
imgIds = coco.getImgIds(catIds=catIds)
img_list = coco.loadImgs(imgIds[12])  # http://images.cocodataset.org/val2017/000000370208.jpg
img = img_list[0]

# plot image
plt.figure(figsize=(10, 10))
pil_img = coco2pil(img["coco_url"])
plt.imshow(pil_img)

# plot boundy boxes
annIds = coco.getAnnIds(imgIds=img["id"])
anns = coco.loadAnns(annIds)
coco.showAnns(anns, draw_bbox=True)
plt.axis('off')
plt.show()
print('Image data: ')
img

In [ ]:
import torchvision.transforms.functional as TF
from PIL import ImageDraw
import matplotlib.pyplot as plt

# lets predict objects by resnet50
with torch.no_grad():
    tensor = TF.pil_to_tensor(pil_img) / 255 # Normalize
    output = fr_rcnn(tensor.unsqueeze(0))
    draw = ImageDraw.Draw(pil_img)
    
    # plot rectangles
    for i, bbox in enumerate(output[0]["boxes"]):
        if output[0]["scores"][i] > 0.5: 
            draw.rectangle((tuple(bbox[:2].numpy()), tuple(bbox[2:].numpy())), width=2)

    plt.figure(figsize=(10, 10))
    plt.imshow(pil_img)
    plt.axis('off')
    plt.show()

### Two stage detector

Faster RCNN = Two stage detector

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/42.png" width="750">

Предложения по объединению ролей

Первый этап: 

Запуск один раз для 
каждого изображения
- Backbone network
- Region Proposal Network
- Region proposal network feature map

Второй этап: 

Запуск один раз для каждого региона
- Особенности обрезки: Rol pool / align
- Предсказать класс объектов CNN
- Prediction box offset

На среднем и верхнем слое выполняются очень похожие операции. Разница в том, что на последнем слое предсказывается класс объекта, а промежуточном только вероятность его присутствия (objectness)

### One Stage detector

Если сразу предсказывать класс, то можно избавиться от второй стадии.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/43.png" width="800">

Детекторы работающие "за один проход":

YOLO, SSD, RetinaNet

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/44.png" width="1000">

[Сравнение скорости моделей](https://pytorch.org/vision/stable/models.html#runtime-characteristics)

#### SSD: Single Shot MultiBox Detector

[SSD: Single Shot MultiBox Detector (Liu et al., 2015)](https://arxiv.org/abs/1512.02325)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/45.png" width="900">

* модель VGG-16, предобученная на ImageNet
* вручную определяем набор соотношений сторон, используемых для B ограничивающих прямоугольников в каждой ячейке сетки, а также смещения (x,y,w,h)
* напрямую предсказывает вероятность того, что класс присутствует в bounding box.
* есть класс для "background"

#### FocalLoss

Retina Net - [Focal Loss for Dense Object Detection (Lin et al., 2017)](https://arxiv.org/abs/1708.02002)



<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/L9-43.png" width="650">

[Блог-пост: Что такое Focal Loss и когда его использовать](https://amaarora.github.io/2020/06/29/FocalLoss.html)

Фокальные потери разработаны для решения одноэтапного сценария обнаружения объектов, когда во время обучения наблюдается крайний дисбаланс между классами переднего и заднего плана (например, 1:1000).

#### Feature pyramid network

[Feature Pyramid Networks for Object Detection (Sergelius et al., 2016)](https://arxiv.org/abs/1612.0314)

Это feature extractor для детекторов.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/18.png" width="650">

Так же, как и в случае с сегментацией, точность повышается если делать предсказания на картах признаков разных масштабов.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/46.png" width="650">

Но первые слои содержат мало семантической информации (только низкоуровневые признаки). Из-за этого детектирование (и сегментация) мелких объектов удается хуже.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/47.png" width="650">

Идея состоит в том, что бы делать предсказание с учетом семантической информации полученной на более глубоких слоях. 


При этом признаки суммируются.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/48.png" width="650">

Затем к новым картам признаков может применяться дополнительная свертка.


На выходе получаем карты признаков P2 - P5 на которых уже предсказываются bounding box.


В случае 2-stage детектора (RCNN) карты подаются на вход RPN 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/49.png" width="850">

А признаки для предсказаний используются из backbone 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/12.jpg" width="1300">

RetinaNet использует выходы FPN и для предсказаний класса и bbox. 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-42.png" width="700">


[Блог-пост про FPN](https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c)

### YOLO

* [You Only Look Once: Unified, Real-Time Object Detection (Redmon et. al., 2015)](https://arxiv.org/abs/1506.02640) 
* [YOLO9000: Better, Faster, Stronger (Redmon et. al., 2015)](https://arxiv.org/abs/1612.08242)
* [YOLOv3: An Incremental Improvement (Redmon et. al., 2018)](https://arxiv.org/abs/1804.02767)
* [YOLOv4: Optimal Speed and Accuracy of Object Detection (Bochkovskiy et al., 2020)](https://arxiv.org/abs/2004.10934)
* Июнь 2020. [YOLOv5 (Glenn Jocher)](https://github.com/ultralytics/yolov5)
* Июль 2021. [YOLOX: Exceeding YOLO Series in 2021 (Ge et al., 2021)](https://arxiv.org/abs/2107.08430)

Первые версии проигрывали конкурентам. Но проект развивался. В настоящий момент это, пожалуй, оптимальный детектор по соотношению качество распознавания/скорость.


#### Дополнительная информация

Старые версии YOLO

###### YOLOv3

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/yolov3.png" width="700">

[YOLO v3: Better, not Faster, Stronger](https://towardsdatascience.com/yolo-v3-object-detection-53fb7d3bfe6b#:~:text=YOLO%20v2%20used%20a%20custom,more%20layers%20for%20object%20detection.&text=First%2C%20YOLO%20v3%20uses%20a,layer%20network%20trained%20on%20Imagenet.)

###### YOLOv4

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/yolov4.jpeg" width="700">

[YOLO v4](https://medium.com/visionwizard/yolov4-version-3-proposed-workflow-e4fa175b902)


- SPP block
- Dense Block
- Больше слоев
- online Augmentation


#### YOLOv5


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-47.png" width="700">


Статья не публиковалась.
Точность сравнима  с v4, но модель определенно лучше упакована.

In [ ]:
catIds = coco.getCatIds(catNms=["person", "bicycle"])
# person and bicycle
imgIds = coco.getImgIds(catIds=catIds)
img_list = coco.loadImgs(imgIds[5])
img = img_list[0]

pil_img = coco2pil(img["coco_url"])
plt.figure(figsize=(10, 10))
plt.imshow(pil_img)

annIds = coco.getAnnIds(imgIds=img["id"])
anns = coco.loadAnns(annIds)
coco.showAnns(anns, draw_bbox=True)
plt.axis('off')
plt.show()
print("Image data:")
img

Загрузка модели с Torch Hub



In [ ]:
import torch

# Load model from torch
model = torch.hub.load("ultralytics/yolov5", "yolov5s", pretrained=True)

Из коробки работает с изображениями в разных форматах и даже url, автоматически меняет размер входного изображения, возвращает объект с результатами ...

In [ ]:
from PIL import Image

# Apply yolov5 model
results = model(pil_img)
results.print() # print predicted results
results.save()  # image on disk

print(f'\nresults.xyxy type: {type(results.xyxy)}\
\nlen(results.xyxy): {len(results.xyxy)}\
\nresults.xyxy[0].shape: {results.xyxy[0].shape}')

results.pandas().xyxy[0]

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# plot predicted results
cv_img = np.array(pil_img)
RGB_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)

annos = results.pandas().xyxy[0]

for i in range(len(annos)):
    x_min, y_min, x_max, y_max = (
        int(annos["xmin"].iloc[i]),
        int(annos["ymin"].iloc[i]),
        int(annos["xmax"].iloc[i]),
        int(annos["ymax"].iloc[i]),
    )

    if annos["name"].iloc[i] == "person":
        color = (255, 255, 255)
    if annos["name"].iloc[i] == "bicycle":
        color = (0, 0, 255)
    if annos["name"].iloc[i] == "backpack":
        color = (0, 255, 0)
    RGB_img = cv2.rectangle(RGB_img, (x_min, y_min), (x_max, y_max), color, 2)
cv2_imshow(RGB_img)

Однако если подать на вход модели тензор, выходы радикально меняются. YOLO переходит в режим обучения, что довольно не очевидно

In [ ]:
input = torch.rand((1, 3, 416, 416))
results = model(input)
print(f'type(results): {type(results)}\nlen(results): {len(results)}\n')
print(f'results[0].shape: {results[0].shape}\n')


## Нard Example Mining

В общем случае, при обучении с учителем набор данных представляет собой набор упорядоченных пар $(\mathbf{x}_i, \mathbf{y}_i)$, целью обучения является аппроксимация функции $f(\mathbf{x}_i) = \mathbf{y}_i$. 

Аппроксимация функции происходит путём уменьшения ошибки предсказания модели. Ожидаемо, модель может показывать "хорошие" результаты на **большей** части данных (изображения с одного и того же ракурса, сделанные при одном освещении), однако "плохо" справляться с нетипичными данными (изображения с другого ракурса, иное освещение). При этом, нетипичные данные не обязательно являются выбросами.

К примеру, рассмотрим задачу обнаружения автомобилей на потоках данных с камер наружного видеонаблюдения. Если в обучающем наборе большая часть данных - снимки, сделанные днём, то качество работы модели ночью будет низким. В данном случае, "нетипичными" данными будут ночные снимки. Но, на самом-то деле, "нетипичных" случаев может быть довольно много, и некоторые из них могут происходить даже днём: 
* ночь (изменение освещения)
* дождь, туман (изменение резкости, помехи на изображении)
* смена сезона (снег либо листья могут покрыть дорогу - изменение фона)
* и другие...

Довольно простым и эффективным решением проблемы является сбор "сложных" случаев (**hard example mining**) и дообучение модели на них. При этом, поскольку модель уже довольно хорошо работает на большей части данных, можно дополнительно удалить часть данных из обучающей выборки - таким образом, мы сосредатачиваем модель на обучении на сложных примерах. 

К примеру, **SVM** - прекрасная модель для обучения на hard example: сложные примеры просто становятся опорными векторами, тогда как "простые примеры" по сути не используются при обучении. Однако давайте заметим, что модель придётся каждый раз переобучать на новом наборе сложных примеров, что довольно затратно.

В различных реальных приложениях компьютерного зрения, довольно часто приходится обращаться к дообучению на сложных примерах (как раз из-за высокой вариативности данных). Однако, существуют и некоторые другие интересные способы борьбы с падением качества работы модели из-за вариативности данных.

Одним из интересных способов является уже упомянутый **focal loss**.  
Суть его заключается в том, что при использовании кросс-энтропии, loss от большого количества "хорошо распознанных" примеров может быть более значительным, чем loss от малого количества "плохо распознанных" примеров. Итого, при обучении модель стремится снизить свою ошибку на большинстве и так уже неплохо предсказываемых примеров, не стремясь исправлять высокую ошибку на редких примерах (что может привести к её дальнейшему увеличению). 

Идея **focal loss** заключается в том, что можно попробовать снизить значения **cross-entropy loss** на итак уже неплохо предсказываемых примерах, чтобы дать модели возможность лучше обучиться на "сложных" для неё примерах. 

Для начала, давайте вспомним формулу кросс-энтропии:

$$p_t = 
\begin{cases}
p &\text{if }y=1\\
1-p &\text{otherwise,}
\end{cases}$$  
$$\text{CE}(p_t) = -\log(p_t).$$

Теперь давайте посмотрим на focal loss:

$$\text{FL}(p_t) = -(1 - p_t)^{\gamma}\log(p_t) = \text{CE}(p_t) \cdot (1 - p_t)^{\gamma}.$$

Отметим, что $(1 - p_t)^{\gamma} = 1$ при $\gamma = 0$, то есть focal loss в какой-то мере является обобщением cross-entropy loss.  
Также, при $\gamma > 0$, множитель является убывающей функцией, принимающей значения ниже единицы. Однако, значение множителя убывает достаточно быстро, чтобы влияние сложных примеров на loss повысилось относительно влияния простых примеров.

Чтобы убедиться в этом, давайте построим графики focal loss при различных значениях $\gamma$ в промежутке $(0; 1)$.

In [ ]:
import numpy as np

def cross_entropy(percent_correct):
    return -np.log(percent_correct)

def focal_loss(percent_correct, gamma = 5):
    return cross_entropy(percent_correct) * (1 - percent_correct)**gamma 

In [ ]:
import plotly.graph_objects as go

eps = 10e-3 # small constant
random_x = np.linspace(eps, 1-eps, 1000)

fig = go.Figure()

gammas = [0] + [i for i in range(1, 20, 4)]
for gamma in gammas:
  fig.add_trace(go.Scatter(x=random_x, y=focal_loss(random_x, gamma),
                      mode='lines',
                      name=str(gamma)))

fig.update_layout(
    title="Focal loss",
    xaxis_title="Correct percent",
    yaxis_title="Loss value",
    legend_title="Value of gamma",
)

fig.show()

Как уже обсуждалось, проблема cross-entropy заключается в том, что большое количество примеров с небольшой ошибкой вносит такой же loss, как и малое количество примеров с большой ошибкой. Давайте попробуем в этом убедиться.

In [ ]:
p1 = 0.8 # вероятность для почти правильных предсказаний
p2 = 0.4 # вероятность для проблемных предсказаний

print(f"При вероятности почти правильных предсказаний {p1} и вероятности проблемных предсказаний {p2}\n")

for gamma in gammas:
    fl1 = focal_loss(p1, gamma)
    fl2 = focal_loss(p2, gamma)
    print(f"Для gamma = {gamma},".ljust(15), f"для равного loss с проблемным предсказанием, почти правильных требуется {int(fl2 / fl1)}")

Как видно, при увеличении значения $\gamma$, в реальности можно достичь значительного роста "важности" примеров с высокой ошибкой, что по сути позволяет модели обращать внимание на "hard examples". 

### Online hard example mining

В некоторых случаях, hard exapmle mining можно выполнять прямо во время формирования батча, "на лету". В таких случаях, говорят про **online hard example mining**. 

Один из вариантов может быть реализован в two-stage детекторах.  
Напоминаю: первая часть детектора вытупает за обнаружение regions of interest, затем выполняется (как правило, сравнительно вычислительно дешёвая) классификация. Одним из вариантов реализации идеи может быть выполнение forward pass классификатора по всем предложенным RoI, и затем формированию батча, в котором будет выделено определённое количество "мест" под RoI, предсказания на которых выполняются наихудшим образом.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-49.png" width="700">

[Блог пост про Hard Mining Example](https://erogol.com/online-hard-example-mining-pytorch/)

[Training Region-based Object Detectors with Online Hard Example Mining (Shrivastava et al., 2016)](https://arxiv.org/abs/1604.03540)

[Loss Rank Mining: A General Hard ExampleMining Method for Real-time Detectors (Yu et al., 2018)](https://arxiv.org/abs/1804.04606)

# Instance Segmentation

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/0.png" width="650">

[COCO panoptic](https://cocodataset.org/#panoptic-2020)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-03.png" width="700">

**Mask R-CNN** (Detectron) - концептуально простая, гибкая и общая схема сегментации объектов. Подход эффективно обнаруживает объекты на изображении и одновременно генерирует высококачественную маску сегментации для каждого объекта. 

Метод, названный Mask R-CNN, расширяет Faster R-CNN (который мы обсуждали ранее), добавляя ветвь для предсказания маски объекта параллельно с существующей ветвью для распознавания *bounding boxes*. 

Код доступен [по ссылке](https://github.com/facebookresearch/Detectron)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/53.png" width="750">

[Модель Mask R-CNN](https://pytorch.org/vision/stable/models.html#mask-r-cnn)

[Пример запуска Mask R-CNN есть в документации Pytorch](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html)



## ROI Align


Выравнивание области интереса, или **RoI Align**, - это операция извлечения небольшой карты признаков из каждого RoI (Region of Interest) в задачах обнаружения и сегментации. Она устраняет жесткое квантование *RoI pool*, правильно выравнивая извлеченные признаки с входными данными. Чтобы избежать квантования границ или бинов RoI, RoIAlign использует билинейную интерполяцию для вычисления точных значений входных признаков в четырех регулярно дискретизированных местах в каждом бине RoI, а затем результат агрегируется (используя max или average).

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/54.png" width="750">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/55-1.png" width="800">

[[doc] torchvision.ops.roi_align](https://pytorch.org/vision/stable/ops.html#torchvision.ops.roi_align)

# Оценка качества детекции

## mAP - mean Average Precision

AP (*Average Precision* - средняя точность) - это популярная метрика для измерения качества детекторов объектов, таких как Faster R-CNN, SSD и др. Средняя точность вычисляет среднее значение *precision* (точности) для значения *recall* от 0 до 1. Звучит сложно, но на самом деле довольно просто. Давайте разберем на конкретных примерах. Но перед этим давайте вкратце разберем что такое *precision*, *recall* и IoU.

### Precision & recall

**Precision** измеряет, насколько точны предсказания сети (т.е. процент правильных предсказаний)

**Recall** измеряет, насколько хорошо сеть находит все положительные срабатывания (*positives*). Например, мы можем найти 80% возможных положительных срабатываний в наших K лучших предсказаниях.

Вот их математические определения:

$\displaystyle\mathrm{Precision} = \frac{TP}{TP+FP}$

$\displaystyle\mathrm{Recall} = \frac{TP}{TP+FN}$

где $TP$ - True Positive, $TN$ - True Negative, $FP$ - False Positive, $FN$ - False Negative.



Например, мы пытаемся детектировать яблоки на фотографиях. Предположим, мы обработали 20 фотографий (на 10 фотографиях по одному яблоку и на 10 фотографиях яблок нет) и обнаружили что:

* в 7 случаях наша нейросеть обнаружила яблоко там, где оно было на самом деле (True Positive),
* в 3 случаях не обнаружила яблоко там, где оно было (False Negative),
* в 4 случаях обнаружила яблоко там, где его не было (False Positive)
* в 6 случаях правильно определила, что на фотографии яблок нет (True Negative),



Посчитаем precision и recall:

In [ ]:
def precision(TP, FP):
    return TP/(TP+FP)

def recall(TP, FN):
    return TP/(TP+FN)

pres = precision(TP=7, FP=4)
rec  = recall(TP=7, FN=3)

print('Precision = %.2f' % pres)
print('Recall = %.2f' % rec)

### IoU (Intersection over union)

IoU измеряет перекрытие между двумя границами. Мы используем его для измерения того, насколько сильно наша предсказанная граница совпадает с истиной (границей реального объекта). В некоторых наборах данных мы заранее определяем порог IoU (например, 0.5) для классификации того, является ли предсказание True Positive или False Positive.

Например, рассмотрим предсказание сети для фотографии яблока:

In [ ]:
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L12/red_apple.jpg -O img.jpg

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

img = Image.open('img.jpg')
array = np.array(img)

fig,ax = plt.subplots(figsize=(5,5))

x0 = 75
y0 = 80
w0 = 450
h0 = 440

ground_truth = Rectangle((x0, y0), w0, h0, linewidth=2, edgecolor='g', linestyle='--', facecolor='none')

x1 = 90
y1 = 120
w1 = 500
h1 = 310
predicted = Rectangle((x1, y1), w1, h1, linewidth=2, edgecolor='b', facecolor='none')

ax.add_patch(ground_truth)
ax.add_patch(predicted)

ax.imshow(array)
ax.axis('off')
plt.show()

IoU определена как

$\displaystyle\mathrm{IoU} = \frac{\text{area of overlap}}{\text{area of union}}$

Посчитаем *area of overlap* и *area of union*:

In [ ]:
def area_of_overlap(x0, y0, x1, y1, w0, w1, h0, h1):
    x0_max = x0 + w0
    y0_max = y0 + h0
    x1_max = x1 + w1
    y1_max = y1 + h1
    dx = min(x0_max, x1_max) - max(x0, x1)
    dy = min(y0_max, y1_max) - max(y0, y1)
    if (dx>=0) and (dy>=0):
        return dx*dy

def area_of_rectangle(w, h):
    return w*h
# Compute intersection areas of the predictions
a_of_overlap = area_of_overlap(x0, y0, x1, y1, w0, w1, h0, h1)

# Compute individual areas of the rectangles
a_0 = area_of_rectangle(w0, h0)
a_1 = area_of_rectangle(w1, h1)

# Compute area of their union
a_of_union = a_0+a_1-a_of_overlap

print('Area of overlap = %i' % a_of_overlap)
print('Area of union = %i' % a_of_union)

Теперь посчитаем IoU

In [ ]:
IoU = a_of_overlap/a_of_union
print('IoU = %.2f' % IoU)

Посмотрим, как будет меняться IoU в зависимости от качества предсказания

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

def plot_predictions_and_calculate_IoU(x1, y1, w1, h1):
    fig,ax = plt.subplots(figsize=(5,5))

    x0 = 75
    y0 = 80
    w0 = 450
    h0 = 440

    ground_truth = Rectangle((x0, y0), w0, h0, linewidth=2, edgecolor='g', linestyle='--', facecolor='none')
    predicted = Rectangle((x1, y1), w1, h1, linewidth=2, edgecolor='b', facecolor='none')

    ax.add_patch(ground_truth)
    ax.add_patch(predicted)

    ax.imshow(array)
    ax.axis('off');

    a_of_overlap = area_of_overlap(x0, y0, x1, y1, w0, w1, h0, h1)
    a_0 = area_of_rectangle(w0, h0)
    a_1 = area_of_rectangle(w1, h1)
    a_of_union = a_0+a_1-a_of_overlap
    IoU = a_of_overlap/a_of_union
    print('IoU = %.2f' % IoU)
    plt.show()

interact(plot_predictions_and_calculate_IoU, 
         x1 = widgets.IntSlider(min=0, max=array.shape[0], step=10, value=90), 
         y1 = widgets.IntSlider(min=0, max=array.shape[1], step=10, value=120), 
         w1 = widgets.IntSlider(min=0, max=array.shape[0], step=10, value=500),
         h1 = widgets.IntSlider(min=0, max=array.shape[1], step=10, value=310)
        );

Видим, что чем лучше предсказание совпадает с реальностью - тем выше у нас значение метрики IoU

### Average Precision

Продолжая тему яблок, возьмем все те же 20 фотографий на 10 из которых яблоки были, а на 10 яблок не было. И представим, что у нас есть некая нейросеть, которая выдает следующие предсказания по всему датасету (или например по батчу):

In [ ]:
import pandas as pd

nn_preds = pd.DataFrame({'IoU' : [0.6, 0.98, 0.4, 0.3, 0.1, 0.96, 0.7, 0.3, 0.2, 0.8],  
                        'precision' : [1,1,0.67,0.5,0.4,0.5,0.57,0.5,0.44,0.5],
                        'recall' : [0.2,0.4,0.4,0.4,0.4,0.6,0.8,0.8,0.8,1]})
nn_preds


Будем считать, что если $\mathrm{IoU} \geq 0.5$ - то предсказание правильное

In [ ]:
nn_preds['correct'] = False
nn_preds['correct'][nn_preds['IoU'] >= 0.5] = True

nn_preds

Можно заметить, что precision имеет зигзагообразный характер - она снижается при ложных срабатываниях и снова повышается при истинных срабатываниях. 

Давайте построим график precision от recall и убедимся

In [ ]:
fig,ax = plt.subplots(figsize=(10,3))
ax.plot(nn_preds.recall, nn_preds.precision)
ax.grid('on')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
plt.show()

По определению, что бы найти AP, нужно найти площадь под кривой recall-precision:

$\displaystyle AP = \int_0^1p(r)dr$

Господи, интеграл! Какая жуть!

Precision и recall всегда находятся в пределах от 0 до 1. Поэтому $AP$ также находится в пределах от 0 до 1. Перед расчетом $AP$ для обнаружения объекта мы часто сначала сглаживаем зигзагообразный рисунок (на каждом уровне recall мы заменяем каждое значение precision максимальным значением точности справа от этого уровня отзыва)

In [ ]:
def smooth_precision(nn_preds):
    smooth_prec = []
    for i in range(len(nn_preds.precision.values)):
        max = nn_preds.precision.values[i:].max()
        smooth_prec.append(max)
    nn_preds['smooth_precision'] = smooth_prec
    return nn_preds

nn_preds = smooth_precision(nn_preds)

Давайте посмотрим как это выглядит на графике

In [ ]:
fig,ax = plt.subplots(figsize=(10,3))
ax.plot(nn_preds.recall, nn_preds.precision, label='precision', color='blue')
ax.plot(nn_preds.recall, nn_preds.smooth_precision, label='smooth precision', color='red')
ax.grid('on')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.legend()
plt.show()

Зачем нам нужно сглаживание? Что бы снизить влияние случайных выбросов и "прыжков" в предсказаниях модели

Теперь давайте посчитаем-таки $AP$

In [ ]:
fig,ax = plt.subplots(figsize=(10,3))
ax.plot(nn_preds.recall, nn_preds.precision, label='precision', color='blue')
ax.plot(nn_preds.recall, nn_preds.smooth_precision, label='smooth precision', color='red')
ax.fill_between(nn_preds.recall, nn_preds.smooth_precision, 
                np.zeros_like(nn_preds.smooth_precision), 
                color='red', alpha=0.1,
                label='Area under the curve')
ax.grid('on')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_ylim(0.35,1.05)
ax.legend()
plt.show()

In [ ]:
from sklearn.metrics import auc

AP = auc(nn_preds.recall, nn_preds.smooth_precision)

print('AP = %.2f' % AP)

### COCO mAP

В последних исследовательских работах, как правило, приводятся результаты только для набора данных COCO. Для COCO AP - это среднее значение (*mean*) по нескольким IoU (минимальный IoU, который следует считать положительным совпадением). AP@[.5:.95] соответствует среднему AP для IoU от 0.5 до 0.95 с шагом 0.05. 

Давайте попробуем посчитать mAP. Для этого посчитаем AP для каждого уровня IoU:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

nn_prediction_at_iou = []
APs = []
for iou in np.arange(0.5,1,0.05):
    nn_preds_limited = nn_preds[nn_preds['IoU'] >= iou]
    nn_preds_limited = smooth_precision(nn_preds_limited)
    AP = auc(nn_preds_limited.recall, nn_preds_limited.smooth_precision)
    APs.append(AP)

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(10, 5), dpi=80)

plt.plot(np.arange(0.5,1,0.05), APs, color='black')
plt.axhline(np.mean(APs), color='red', ls='--', label='mAP')
plt.xlabel('IoU')
plt.ylabel('AP')
plt.grid('on')
plt.legend();

print('mAP@[0.5:0.95] = %.2f' % np.mean(APs))
plt.show()

Есть несколько различных определений mAP, которые разняться от соревнования к соревнованию (суть одинаковая, но разница в деталях подхода), поэтому для каждого соревнования лучше использовать их собственные библиотеки для расчета

# DINO — Self-supervised representation learning (with segmentation capabilities)
[Emerging Properties in Self-Supervised Vision Transformers (Caron et al., 2021)](https://arxiv.org/abs/2104.14294)

[Отличное видео объяснение статьи](https://www.youtube.com/watch?v=h3ij3F3cPIk)

Для начала подгрузим модель DINO (self-**DI**stillation with **NO** labels)

*Перед запуском необходимо сбросить среду выполнения!

In [ ]:
# fix random_seed
import torch
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Compute on cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!git clone https://github.com/facebookresearch/dino.git

Теперь загрузим случайную картинку (можно выбрать любую, просто замените ссылку на свою)

In [ ]:
from PIL import Image

URL = 'https://edunet.kea.su/repo/EduNet-web_dependencies/L12/capybara_image.jpg'
!wget $URL -qO test.jpg

## Результаты DINO

И посмотрим, что с ней может сделать DINO, а затем обсудим как это работает и что вообще происходит

In [ ]:
from IPython.display import clear_output
!python /content/dino/visualize_attention.py --image_path /content/test.jpg 
clear_output()

Пока не вдаваясь в детали, посмотрим на картинки которые генерирует DINO

In [ ]:
from glob import glob
import matplotlib.pyplot as plt

def img_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols
    fig,ax = plt.subplots(nrows=rows, ncols=cols, figsize=(28,8))
    for num, img in enumerate(imgs):
        img_PIL = Image.open(img)
        ax[num].imshow(img_PIL)
        ax[num].set_xticks([])
        ax[num].set_yticks([])
    plt.subplots_adjust(hspace=0, wspace=0)

img_grid(imgs=sorted(glob('*.png'))[::-1], rows=1, cols=7)
plt.show()

## Принцип работы 

Что бы понять, что мы видим, давайте разберем архитектуру DINO и посмотрим, как ее обучали. На самом деле, DINO не столько архитектура, сколько метод - то есть в качестве backbone можно использовать любую нейросеть (например, ResNet или ViT). Самые лучшие результаты показали DINO на основе VIT, соответственно в этом блокноте будем разбирать именно эту конфигурацию. Также для понимания принципа работы DINO, кратко разберем, что такое дистилляция знаний ([knowledge distillation](https://arxiv.org/abs/2006.05525)) и как применяется. 

Для начала, давайте вспомним как работает ViT. **Vi**sual **T**ransformer получает на вход картинку, разбитую на кусочки (*patches*) размером 8x8 или 16x16 пикселей. Затем эти кусочки расправляются в вектор (*flatten*) и пропускаются через энкодер трансформера. Поверх трансформера прикручена MLP голова, которая собирает информацию с голов трансформера и предсказывает класс для картинки.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/l12out2.png" width="700">

[vision_transformer](https://github.com/google-research/vision_transformer)

Разберемся с понятием дистилляция знаний нейронных сетей. 

Представим, что у нас есть обученная сеть с большим количеством параметров (весов) и мы хотели бы использовать ее с меньшим количеством параметров (получить более легковесную версию сети, например для использования на мобильном устройстве). Один из подходов - обучить маленькую модель имитировать предсказания большой модели. Т.е., мы можем попробовать на предсказаниях большой модели (*teacher*), обучить легковесную модель (*student*).

Идея в том, что большие модели содержат больше информации (обладают бОльшим знанием), но не вся эта информация используется, поэтому с помощью дистилляции мы сможем вытащить наиболее важную информацию для предсказаний.

 <img src ="https://edunet.kea.su/repo/EduNet-content/L12/img/distillation.png" width="700">

DINO - это self-supervised метод, а значит классы ему недоступны. Что же делает эта сеть?

Разберем по шагам. DINO на вход получает изображение $x$.

In [ ]:
input_img = Image.open('/content/img.png')
input_img

Дальше к этому изображению применяются две различные аугментации - $x_1$ и $x_2$

In [ ]:
import torchvision
from torchvision import transforms

transform_x1 = transforms.Compose(
    [
        transforms.RandomCrop(size=(128, 128)),
        transforms.ColorJitter(
            brightness = np.random.uniform(0.5,1),
            contrast = np.random.uniform(0.5,1),
            saturation = np.random.uniform(0.5,1)
        ),
        transforms.GaussianBlur((1,5),(0.1,2))
    ]
)

transform_x2 = transforms.Compose(
    [
        transforms.RandomCrop(size=(128, 128)),
        transforms.ColorJitter(
            brightness = np.random.uniform(0.5,1),
            contrast = np.random.uniform(0.5,1),
            saturation = np.random.uniform(0.5,1)
        ),
        transforms.GaussianBlur((1,3),(0.1,1))     
    ]
)


aug_image_x1 = transform_x1(input_img)
aug_image_x2 = transform_x2(input_img)

fig,ax = plt.subplots(ncols=2,figsize=(10,20))
ax[0].imshow(aug_image_x1)
ax[0].axis('off')
ax[1].imshow(aug_image_x2)
ax[1].axis('off')
plt.show()

Далее каждая из этих аугментаций проходит через свою собственную версию **ViT** - *student* и *teacher*. Однако, в DINO, teacher и student имеют похожую архитектуру (учитель не является большой сетью, а студент маленькой). 

Обучается именно студент, а веса учителя обновляются как **E**xponential **M**oving **A**verage (ema) весов студента.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img_licence/65.png" width="450">

На схеме можно заменить, что используется *centering* при прогоне через учителя, фактически это позволяет держать значения в диапозоне (похоже на нормализацию), чтобы избежать коллапса. 

В конечном итоге обе ветки сети выдают какое-то представление (*representation*) данных, которое, как мы надеемся, будет близко для похожих изображений и далеко для непохожих. Давайте на него посмотрим

In [ ]:
import torch
import sys
mod_name = 'utils'
if mod_name in sys.modules:
    del sys.modules[mod_name]
    
vits16 = torch.hub.load('facebookresearch/dino:main', 'dino_vits16', force_reload=True, skip_validation=True)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

transform = transforms.ToTensor()

# Transform each augmented image into a tensor
x1_tensor = transform(aug_image_x1).unsqueeze(0) 
x2_tensor = transform(aug_image_x2).unsqueeze(0)

# Get representation from DINO
x1_representation = vits16(x1_tensor)
x2_representation = vits16(x2_tensor)

fig, ax = plt.subplots(ncols=2, figsize=(10,5))
ax[0].imshow(x1_representation.view(24,16).detach().cpu().numpy())
ax[1].imshow(x2_representation.view(24,16).detach().cpu().numpy())
ax[0].set_title('$x_1$');
ax[1].set_title('$x_2$');
plt.show()

Выглядит и впрямь довольно похоже. А что если мы засунем туда что-то совершенно непохожее?

In [ ]:
URL = 'https://edunet.kea.su/repo/EduNet-content/L12/img_license/robots.jpg'
!wget $URL -qO robots.jpg

input_img = Image.open('/content/robots.jpg')
input_img

In [ ]:
transform_x3 = transforms.Compose(
    [
     transforms.RandomCrop(size=(128, 128)),
     transforms.RandomSolarize(0.5, p=1)
     ]
    )

aug_image_x3 = transform_x3(input_img)

aug_image_x3

In [ ]:
x3_tensor = transform(aug_image_x3).unsqueeze(0) 

x3_representation = vits16(x3_tensor[:,:3,:,:])

fig, ax = plt.subplots(ncols=3, figsize=(10,5))
ax[0].imshow(x1_representation.view(24,16).detach().cpu().numpy())
ax[1].imshow(x2_representation.view(24,16).detach().cpu().numpy())
ax[2].imshow(x3_representation.view(24,16).detach().cpu().numpy())
ax[0].set_title('$x_1$');
ax[1].set_title('$x_2$');
ax[2].set_title('$x_3$');
plt.show()

Да, выглядит не очень похоже

Loss для такой сети можно записать следующим образом: 

$\text{loss} = H(t_1, s_2)/2 + H(t_2, s_1)/2$,

где, $ H(a, b) = −a \space log(b)$

 $t_i$ - выученное представление i-ой аугментации teacher network и $s_i$ -  выученное представление i-ой аугментации student network

## Сегментация изображений

Давайте вновь посмотрим на результаты

In [ ]:
img_grid(imgs=sorted(glob('*.png'))[::-1], rows=1, cols=7)

Мы видим 6 карт внимания (*self-attention maps* - веса слоя self-attention) на 6 головах Visual Transformer. В результате self-supervised обучения по методике DINO, трансформер **САМОСТОЯТЕЛЬНО** придумал обращать внимание на различные части изображения, таким образом, производя семантическую сегментацию.

Мы можем все эти карты внимания объединить в одно изображение, и просто назначить каждой карте свой цвет, а в качестве прозрачности использовать интенсивность

In [ ]:
import matplotlib.colors as mcolors
import numpy as np
from matplotlib import cm

def overlay(img, segmentations):
    img_PIL = Image.open(img)
    fig, ax = plt.subplots(ncols=2, figsize=(10,10))
    ax[0].imshow(img_PIL)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    
    ax[1].imshow(img_PIL.convert('LA'), alpha=0.5)
    for num, img in enumerate(segmentations):
        segment_PIL = Image.open(img).convert('LA')
        segment_arr = np.array(segment_PIL)
        colors = [(*cm.tab10(num)[:-1], c) for c in np.linspace(0,0.75,100)]
        cmap = mcolors.LinearSegmentedColormap.from_list('mycmap', colors, N=5)
        ax[1].imshow(segment_arr[:,:,0], cmap=cmap)
    ax[1].set_facecolor('black')
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    plt.subplots_adjust(hspace=0, wspace=0)

overlay('/content/img.png', sorted(glob('*.png'))[:-1])
plt.show()

Видим, что DINO сегментирует разные части нашей картинки на разные семантические группы. В случае с капибарой - это голова, лицо (нос, глаза) и тело.

## Сегментация видео

Вы думали что на этом все? Нет, DINO может еще удивить. Например она умеет сегментировать видео

In [ ]:
!sudo pip install --upgrade youtube_dl

Скачаем видео (можно любое)

In [ ]:
URL = 'https://edunet.kea.su/repo/EduNet-web_dependencies/L12/cheetah_video.mp4'
!youtube-dl -o video.mp4 $URL

И сгенерируем сегментированное видео

In [ ]:
from IPython.display import clear_output
!python /content/dino/video_generation.py --input_path /content/video.mp4 --output_path  /content/video_segmented --resize 360 640
!ffmpeg -i /content/video.mp4 -vf scale=640:360 /content/video_scaled.mp4
!ffmpeg \
  -i /content/video_scaled.mp4 \
  -i /content/video_segmented/video.mp4 \
  -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
  -map '[vid]' \
  -c:v libx264 \
  -crf 23 \
  -preset veryfast \
  output.mp4

clear_output()
print('Complete')

Посмотрим, что получилось. Обратите внимание, эта сеть обучалась в режиме self-supervision!

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Кластеризация

А еще DINO умеет кластеризовать изображения. Выполнять не будем, так как процесс не быстрый, но можем посмотреть на результаты из их статьи

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/1.gif" width="400">

[Advancing the state of the art in computer vision with self-supervised Transformers and 10x more efficient training](https://ai.facebook.com/blog/dino-paws-computer-vision-with-self-supervised-transformers-and-10x-more-efficient-training/)

<font size ="6">Список использованной литературы

 COCO

[Подробнее о том как создать свой COCO датасет с нуля](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch).

[Подробнее о разметке COCO](https://cocodataset.org/#format-data)

[Что такое  run-length encoding - RLE](https://en.wikipedia.org/wiki/Run-length_encoding)?

[Видео-разбор Run-length encoding](https://www.youtube.com/watch?v=h6s61a_pqfM)

 Семантическая сегментация

[FCN Semantic segmenation](https://towardsdatascience.com/review-fcn-semantic-segmentation-eb8c9b50d2d1)

[Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation (Chen et al., 2018)](https://arxiv.org/abs/1802.02611v3)

[Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition (He et al., 2014)](https://arxiv.org/abs/1406.4729)

 Детектирование

[Recent Progress in Appearance-based Action Recognition (Humphreys et al., 2020)](https://arxiv.org/abs/2011.12619)

[Статья про Selective Search](http://www.huppelen.nl/publications/selectiveSearchDraft.pdf)

[SSD: Single Shot MultiBox Detector (Liu et al., 2015)](https://arxiv.org/abs/1512.02325)

[Focal Loss for Dense Object Detection (Lin et al., 2017)](https://arxiv.org/abs/1708.02002)

[Блог-пост: Что такое Focal Loss и когда его использовать](https://amaarora.github.io/2020/06/29/FocalLoss.html)

[Feature Pyramid Networks for Object Detection (Sergelius et al., 2016)](https://arxiv.org/abs/1612.0314)

[Understanding feature pyramid networks for object detection - FPN](https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c)

[You Only Look Once: Unified, Real-Time Object Detection (Redmon et. al., 2015)](https://arxiv.org/abs/1506.02640) 

[YOLO9000: Better, Faster, Stronger (Redmon et. al., 2015)](https://arxiv.org/abs/1612.08242)

[YOLOv3: An Incremental Improvement (Redmon et. al., 2018)](https://arxiv.org/abs/1804.02767)

[YOLOv4: Optimal Speed and Accuracy of Object Detection (Bochkovskiy et al., 2020)](https://arxiv.org/abs/2004.10934)

[YOLOv5 (Glenn Jocher)](https://github.com/ultralytics/yolov5)

[YOLOX: Exceeding YOLO Series in 2021 (Ge et al., 2021)](https://arxiv.org/abs/2107.08430)

 Hard example mining

[Блог пост про Hard Mining Example](https://erogol.com/online-hard-example-mining-pytorch/)

[Training Region-based Object Detectors with Online Hard Example Mining (Shrivastava et al., 2016)](https://arxiv.org/abs/1604.03540)

[Loss Rank Mining: A General Hard ExampleMining Method for Real-time Detectors (Yu et al., 2018)](https://arxiv.org/abs/1804.04606)

 DINO 

[Emerging Properties in Self-Supervised Vision Transformers (Caron et al., 2021)](https://arxiv.org/abs/2104.14294)

[Отличное видео объяснение статьи DINO](https://www.youtube.com/watch?v=h3ij3F3cPIk)

 Другое

[U-Net: Convolutional Networks for Biomedical Image Segmentation (Ronneberger et al., 2015)](https://arxiv.org/abs/1505.04597)

[Блог-пост про 2d свертки с помощью перемножения матриц](https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544)